In [1]:
import prosperity3bt
import pandas as pd
import itertools
import numpy as np

from prosperity3bt.runner import run_backtest
from prosperity3bt.models import TradeMatchingMode
from prosperity3bt.file_reader import FileSystemReader
from pathlib import Path

from importlib import reload

In [2]:
def day_profit(result):
    last_timestamp = result.activity_logs[-1].timestamp

    profit = 0
    for row in reversed(result.activity_logs):
        if row.timestamp != last_timestamp:
            break

        profit += row.columns[-1]

    return profit

def backtest_days(trader, days=[-2, -1, 0], data_path='.', round_num=1):
    profits = []
    for day in days:
        backtest_results = prosperity3bt.runner.run_backtest(trader, file_reader=FileSystemReader(Path(data_path)), round_num=round_num, day_num=day, print_output=False, trade_matching_mode=TradeMatchingMode.all, no_names=True, show_progress_bar=False)
        profit = day_profit(backtest_results)
        profits.append(profit)
    return profits

def generate_param_combinations(param_grid):
    param_names = param_grid.keys()
    param_values = param_grid.values()
    combinations = list(itertools.product(*param_values))
    return [dict(zip(param_names, combination)) for combination in combinations]

In [3]:
import ink_trader
reload(ink_trader)

param_grid = {
    # 'ink_change_threshold_pct': [0.01, 0.012, 0.015, 0.018, 0.02, 0.022, 0.025, 0.03],
    'ink_change_threshold_pct': [0.01, 0.012, 0.015, 0.02, 0.022],
    'ink_window_size': [3, 5, 10, 12, 15, 20, 25],
    'ink_position_limit': [50],
    # 'ink_position_limit': [30, 35, 40, 45, 50],
    'clear_price_thresh': [0, 1, 2, 3, 5]
}

PARAMS = {
    'ink_change_threshold_pct': 0.02,
    'ink_window_size': 10,
    'ink_position_limit': 50,
}

param_profits = []

for params in generate_param_combinations(param_grid):
    trader = ink_trader.Trader(params)
    days = [-2, -1, 0]
    profits = backtest_days(trader, days=days, data_path='.', round_num=1)
    for d in days:
        params[f'profit_day_{d}'] = profits[days.index(d)]
    param_profits.append(params)

In [ ]:
import ink_trader
from functools import partial
import concurrent.futures
from itertools import product

def generate_param_combinations(param_grid):
    """Generate all combinations of parameters from the param_grid"""
    keys = param_grid.keys()
    values = param_grid.values()
    combinations = []
    
    for combo in product(*values):
        combinations.append(dict(zip(keys, combo)))
    
    return combinations

def run_backtest(params, days, data_path, round_num):
    """Run backtest with given parameters and return results"""
    trader = ink_trader.Trader(params)
    profits = backtest_days(trader, days=days, data_path=data_path, round_num=round_num)
    
    result = params.copy()
    for d in days:
        result[f'profit_day_{d}'] = profits[days.index(d)]
    
    return result

def parallel_backtest(param_grid, days=[-2, -1, 0], data_path='.', round_num=1, max_workers=None):
    """Run backtests in parallel using ThreadPoolExecutor"""
    param_combinations = generate_param_combinations(param_grid)
    
    # Create a partial function with fixed arguments
    backtest_fn = partial(run_backtest, days=days, data_path=data_path, round_num=round_num)
    
    param_profits = []
    
    # Run in parallel using ThreadPoolExecutor
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(backtest_fn, params): params for params in param_combinations}
        
        for future in concurrent.futures.as_completed(futures):
            try:
                result = future.result()
                param_profits.append(result)
            except Exception as e:
                print(f"Error with params {futures[future]}: {e}")
    
    return param_profits

# Example usage
param_grid = {
    'ink_change_threshold_pct': [0.01, 0.012, 0.015, 0.018, 0.02, 0.022],
    'ink_window_size': [3, 5, 6, 8, 10, 12, 15, 20, 25],
    'ink_position_limit': [30, 40, 50],
    'clear_price_thresh': [0, 1, 2, 3, 5]
}

# Run the parallel backtest
param_profits = parallel_backtest(param_grid)

[[4400,"{\"ink_price_hist\": [2000.0, 2000.5, 2000.5], \"ink_trigger_hist\": [0, 0, 0], \"ink_position_hist\": [0, 0, 0]}",[["KELP","KELP",1],["RAINFOREST_RESIN","RAINFOREST_RESIN",1],["SQUID_INK","SQUID_INK",1]],{"KELP":[{"1999":29},{"2001":-2,"2002":-29}],"RAINFOREST_RESIN":[{"9998":8,"9996":1,"9995":28},{"10004":-1,"10005":-28}],"SQUID_INK":[{"1998":29},{"2000":-2,"2001":-29}]},[],[["KELP",1999,1,"","",4200],["RAINFOREST_RESIN",10004,2,"","",4200],["RAINFOREST_RESIN",10004,1,"","",4200],["SQUID_INK",1999,1,"","",4200]],{},[{},{}]],[],1,"{\"ink_price_hist\": [2000.0, 2000.5, 2000.5], \"ink_trigger_hist\": [0, 0, 0], \"ink_position_hist\": [0, 0, 0]}",""][[20100,"{\"ink_price_hist\": [1996.5, 1996.5, 1996.0], \"ink_trigger_hist\": [0, 0, 0], \"ink_position_hist\": [0, 0, 0]}",[["KELP","KELP",1],["RAINFOREST_RESIN","RAINFOREST_RESIN",1],["SQUID_INK","SQUID_INK",1]],{"KELP":[{"1999":30},{"2001":-7,"2002":-30}],"RAINFOREST_RESIN":[{"10002":1,"9995":30},{"10005":-30}],"SQUID_INK":[{"1994"

In [4]:
np.prod([len(v) for v in param_grid.values()])

np.int64(175)

In [ ]:
df = pd.DataFrame(param_profits)
df['tot_profit'] = df[[f'profit_day_{d}' for d in days]].sum(axis=1)
df.sort_values(by='tot_profit', ascending=False, inplace=True)


In [6]:
df

,ink_change_threshold_pct,ink_window_size,ink_position_limit,clear_price_thresh,profit_day_-2,profit_day_-1,profit_day_0,tot_profit
0,0.010,3,50,0,4273.0,6257.0,0.0,10530.0
1,0.010,3,50,1,4273.0,6257.0,0.0,10530.0
100,0.015,25,50,0,1127.0,8548.0,706.0,10381.0
104,0.015,25,50,5,1127.0,8548.0,706.0,10381.0
103,0.015,25,50,3,1127.0,8548.0,706.0,10381.0
...,...,...,...,...,...,...,...,...
34,0.010,25,50,5,-10156.0,4996.0,-1865.0,-7025.0
33,0.010,25,50,3,-10156.0,4996.0,-1865.0,-7025.0
32,0.010,25,50,2,-10156.0,4996.0,-1865.0,-7025.0
30,0.010,25,50,0,-10156.0,4996.0,-1865.0,-7025.0


In [10]:
df.head(20)

,ink_change_threshold_pct,ink_window_size,ink_position_limit,clear_price_thresh,profit_day_-2,profit_day_-1,profit_day_0,tot_profit
0,0.010,3,50,0,4273.0,6257.0,0.0,10530.0
1,0.010,3,50,1,4273.0,6257.0,0.0,10530.0
100,0.015,25,50,0,1127.0,8548.0,706.0,10381.0
104,0.015,25,50,5,1127.0,8548.0,706.0,10381.0
103,0.015,25,50,3,1127.0,8548.0,706.0,10381.0
102,0.015,25,50,2,1127.0,8548.0,706.0,10381.0
101,0.015,25,50,1,1127.0,8548.0,706.0,10381.0
2,0.010,3,50,2,4273.0,6082.0,0.0,10355.0
3,0.010,3,50,3,4273.0,6082.0,0.0,10355.0
4,0.010,3,50,5,4273.0,6082.0,0.0,10355.0


In [13]:
df[(df['profit_day_-1'] > 0) & (df['profit_day_0'] > 0) & (df['profit_day_-2'] > 0)].sort_values(by='tot_profit', ascending=False)

,ink_change_threshold_pct,ink_window_size,ink_position_limit,clear_price_thresh,profit_day_-2,profit_day_-1,profit_day_0,tot_profit
100,0.015,25,50,0,1127.0,8548.0,706.0,10381.0
104,0.015,25,50,5,1127.0,8548.0,706.0,10381.0
103,0.015,25,50,3,1127.0,8548.0,706.0,10381.0
102,0.015,25,50,2,1127.0,8548.0,706.0,10381.0
101,0.015,25,50,1,1127.0,8548.0,706.0,10381.0
95,0.015,20,50,0,1127.0,3576.0,418.5,5121.5
99,0.015,20,50,5,1127.0,3576.0,418.5,5121.5
96,0.015,20,50,1,1127.0,3576.0,418.5,5121.5
98,0.015,20,50,3,1127.0,3576.0,418.5,5121.5
97,0.015,20,50,2,1127.0,3576.0,418.5,5121.5


In [8]:
df.iloc[0].drop([f'profit_day_{day}' for day in days] + ['tot_profit']).to_dict()

{'ink_change_threshold_pct': 0.01,
 'ink_window_size': 3.0,
 'ink_position_limit': 50.0,
 'clear_price_thresh': 0.0}

In [7]:
df.to_csv('backtests/big_test_results.csv')

In [ ]:
# df.to_csv('backtests/inktrader_bt_results.csv')